In [1]:
# Multiclass classification using a neural network

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import seaborn as sns
import pandas as pd


In [2]:
df_training = pd.read_csv('./dataset/datasetC.csv', header=None)
df_testing = pd.read_csv('./dataset/datasetCTest.csv', header=None)

# X is all the data except the last column
X_training = df_training.iloc[:, :-1]

# y is the last column
y_training = df_training.iloc[:, -1]

In [3]:
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
# Initialization of models
models = {  
    "xgboost": OneVsRestClassifier(XGBClassifier()),
    "svc" : OneVsRestClassifier(SVC()),
}

from sklearn.model_selection import train_test_split

y_training = y_training - 1
x_train, x_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2, random_state=42)

# Scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [5]:
# Train the models
for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(x_train_scaled, y_train)

  #Make predictions
  y_test_pred = model.predict(x_test_scaled)

  # Test set Performance
  accuracy = accuracy_score(y_test, y_test_pred)

  print(list(models.keys())[i])

  print('- Accuracy : {:.4f}'.format(accuracy))

  print('\n')


xgboost
- Accuracy : 0.7970


svc
- Accuracy : 0.8330




In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Assuming y_train are your labels
# Convert labels to categorical one-hot encoding
y_train_categorical = to_categorical(y_train)

# Create a Sequential model
model_nn = Sequential()

# Add an input layer and a hidden layer with 10 neurons
model_nn.add(Dense(10, input_dim=x_train_scaled.shape[1], activation='relu'))

# Add an output layer with a neuron for each class, using softmax activation
model_nn.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Compile the model
model_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model_nn.fit(x_train_scaled, y_train_categorical, epochs=50, batch_size=10)

# Evaluate the model
loss, accuracy = model_nn.evaluate(x_test_scaled, to_categorical(y_test))

print('Test accuracy:', accuracy)

Epoch 1/50
400/400 [==============================] - 2s 3ms/step - loss: 1.2298 - accuracy: 0.5502
Epoch 2/50
400/400 [==============================] - 1s 3ms/step - loss: 0.6135 - accuracy: 0.7915
Epoch 3/50
400/400 [==============================] - 1s 3ms/step - loss: 0.4353 - accuracy: 0.8505
Epoch 4/50
400/400 [==============================] - 1s 3ms/step - loss: 0.3476 - accuracy: 0.8820
Epoch 5/50
400/400 [==============================] - 1s 3ms/step - loss: 0.2884 - accuracy: 0.9040
Epoch 6/50
400/400 [==============================] - 1s 3ms/step - loss: 0.2448 - accuracy: 0.9185
Epoch 7/50
400/400 [==============================] - 1s 3ms/step - loss: 0.2109 - accuracy: 0.9340
Epoch 8/50
400/400 [==============================] - 1s 3ms/step - loss: 0.1799 - accuracy: 0.9470
Epoch 9/50
400/400 [==============================] - 1s 3ms/step - loss: 0.1544 - accuracy: 0.9582
Epoch 10/50
400/400 [==============================] - 2s 4ms/step - loss: 0.1337 - accuracy: 0.9638